In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"amankumarchahar","key":"fd31c625ef31e544aca1189d9663d09d"}'}

In [2]:
# creating path for kaggle file
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [3]:
#API to fetch the dataset from kaggle
!kaggle datasets download -d everydaycodings/global-news-dataset

100% 400M/400M [00:21<00:00, 23.2MB/s]
100% 400M/400M [00:21<00:00, 19.4MB/s]


In [4]:
# extracting a compressed dataset

from zipfile import ZipFile
dataset = '/content/global-news-dataset.zip'

with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


In [5]:
#importing required libraries
import pandas as pd
import numpy as np
import re
import spacy
import string
import nltk
import nltk.tokenize
from tqdm import tqdm_notebook as tqdm
from nltk.corpus import stopwords
from tqdm import tqdm_notebook as tqdm
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [6]:
# Download NLTK resources
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
#Reading Dataset
data = pd.read_csv("/content/rating.csv")
data.head(10)

,article_id,source_id,source_name,author,title,description,url,url_to_image,published_at,content,category,article,title_sentiment
0,81664,NaN,Forbes,"Elizabeth Brownfield, Contributor, \n Elizabet...",superstar chef yannick alléno brings refined f...,Now open in Mayfair at Four Seasons Hotel Lond...,https://www.forbes.com/sites/elizabethbrownfie...,https://imageio.forbes.com/specials-images/ima...,2023-11-01 03:27:21.000000,"Pavyllon London, at Four Seasons Hotel London ...",Monaco,"pavyllon london, at four seasons hotel london ...",Neutral
1,81667,NaN,CNA,NaN,nice claim top spot in ligue 1 with late win a...,Nice moved into provisional first place in the...,https://www.channelnewsasia.com/sport/nice-cla...,https://onecms-res.cloudinary.com/image/upload...,2023-10-27 21:28:48.000000,Nice moved into provisional first place in the...,Monaco,nice moved into provisional first place in the...,Positive
2,81694,time,Time,Christina Larson / AP,amphibians are the world’s most vulnerable spe...,"The world’s frogs, salamanders, newts, and oth...",https://time.com/6320467/amphibians-most-vulne...,https://api.time.com/wp-content/uploads/2023/1...,2023-10-04 17:36:18.000000,"The worlds frogs, salamanders, newts and other...",Madagascar,"the world’s frogs, salamanders, newts and othe...",Negative
3,81695,NaN,Phys.Org,Sara Schmidt,image: rusty red waters in madagascar,Iron-rich sediment colors the red-orange water...,https://phys.org/news/2023-10-image-rusty-red-...,https://scx2.b-cdn.net/gfx/news/2023/image-rus...,2023-10-31 18:04:02.000000,Iron-rich sediment colors the red-orange water...,Madagascar,iron-rich sediment colors the red-orange water...,Neutral
4,81703,NaN,Digital Trends,Jason Struss,everything leaving max (formerly hbo max) in n...,From Gangs of London to Fear the Walking Dead ...,https://www.digitaltrends.com/movies/everythin...,https://www.digitaltrends.com/wp-content/uploa...,2023-10-23 23:09:18.000000,"Everything ends. No, I’m not having an existen...",Madagascar,"everything ends. no, i’m not having an existen...",Neutral
5,81707,al-jazeera-english,Al Jazeera English,Al Jazeera,unga calls for humanitarian truce in israel-ha...,The resolution passed with 120 votes in favour...,https://www.aljazeera.com/news/2023/10/27/unga...,https://www.aljazeera.com/wp-content/uploads/2...,2023-10-27 22:16:05.000000,The United Nations General Assembly has passed...,Madagascar,the resolution passed with 120 votes in favour...,Neutral
6,81717,bbc-news,BBC News,https://www.facebook.com/bbcnews,malawi heatwave warning issued as temperatures...,"Temperatures could soar to 44C, the weather bu...",https://www.bbc.co.uk/news/world-africa-67086254,https://ichef.bbci.co.uk/news/1024/branded_new...,2023-10-12 10:40:48.000000,Malawians have been warned of an extreme heatw...,Madagascar,malawians have been warned of an extreme heatw...,Neutral
7,81719,NaN,Phys.Org,Carlyn Chatfield,researcher scans tropical forest with mixed-re...,Rice University scientists used a commercially...,https://phys.org/news/2023-10-scans-tropical-f...,https://scx2.b-cdn.net/gfx/news/2023/rice-rese...,2023-10-19 20:02:03.000000,Rice University scientists used a commercially...,Madagascar,rice university scientists used a commercially...,Neutral
8,81723,NaN,Deadline,Patrick Hipes,"‘taylor swift: the eras tour’ movie: tickets, ...",Taylor Swift has the highest-grossing concert ...,https://deadline.com/feature/taylor-swift-eras...,https://deadline.com/wp-content/uploads/2023/1...,2023-10-11 14:30:00.000000,Taylor Swift has the highest-grossing concert ...,Madagascar,taylor swifthas the highest-grossing concert t...,Neutral
9,81727,NaN,Euronews,NaN,marine le pen found guilty of defamation after...,Marine Le Pen was convicted of defamation agai...,https://www.euronews.com/2023/10/13/marine-le-...,https://static.euronews.com/articles/stories/0...,2023-10-13 15:34:20.000000,Marine Le Pen was convicted of defamation agai...,Madagascar,"marine le pen, former leader of france's far-r...

In [8]:
#Dataset Information
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58356 entries, 0 to 58355
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   article_id       58356 non-null  int64 
 1   source_id        17771 non-null  object
 2   source_name      58356 non-null  object
 3   author           56193 non-null  object
 4   title            58356 non-null  object
 5   description      58346 non-null  object
 6   url              58356 non-null  object
 7   url_to_image     54905 non-null  object
 8   published_at     58356 non-null  object
 9   content          58356 non-null  object
 10  category         58335 non-null  object
 11  article          58356 non-null  object
 12  title_sentiment  58356 non-null  object
dtypes: int64(1), object(12)
memory usage: 5.8+ MB


# Data Visualisation

In [13]:
from plotly.express import bar

source_counts = data['source_name'].value_counts()
source_counts_df = source_counts.to_frame().reset_index()
source_counts_df.columns = ['source_name', 'count']
top_sources = source_counts_df.sort_values(by='count', ascending=False).head(20)
bar(data_frame=top_sources, x='source_name', y='count', title='Top 20 sources by article count')

# Data Cleaning

In [14]:
# Drop non useful columns
data = data.drop(columns=["article_id", "source_id", "source_name", "author", "description", "url", "url_to_image", "published_at", "content", "category", "article"])

In [15]:
# Drop duplicate title
data = data.drop_duplicates(subset=['title'], keep='first')

In [16]:
# Explore distribution of Title categories
title_sentiment = data["title_sentiment"].value_counts()
print(title_sentiment)

Neutral     39202
Negative     8505
Positive     5747
Name: title_sentiment, dtype: int64


In [17]:
# Handling missing values
data = data.drop_duplicates()

In [18]:
# Drop rows with missing values
data = data.dropna()

In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53454 entries, 0 to 58355
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   title            53454 non-null  object
 1   title_sentiment  53454 non-null  object
dtypes: object(2)
memory usage: 1.2+ MB


In [20]:
#Balance Dataset
neutral_rows = data[data['title_sentiment'] == 'Neutral']
neutral_subset = neutral_rows.head(6000)
positive_rows = data[data['title_sentiment'] == 'Positive']
negative_rows = data[data['title_sentiment'] == 'Negative']
data = pd.concat([neutral_subset, positive_rows, negative_rows], ignore_index=True)

In [21]:
data["title_sentiment"].value_counts()

Negative    8505
Neutral     6000
Positive    5747
Name: title_sentiment, dtype: int64

# Sementic search using transformers

In [36]:
! pip install faiss-cpu
! pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 668.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 886.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [41]:
from sentence_transformers import SentenceTransformer

text = data['title']
encoder = SentenceTransformer("paraphrase-mpnet-base-v2") # bert-base-nli-mean-tokens
embeddings = encoder.encode(text)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning:


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.



modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [42]:
embeddings

array([[ 0.01464342,  0.23955193,  0.06381194, ...,  0.06828717,
         0.02958326, -0.02240116],
       [-0.03478733, -0.14131786, -0.02610976, ..., -0.00062566,
        -0.08203534,  0.04742828],
       [-0.19414957,  0.09962042,  0.07084082, ...,  0.05764551,
         0.03674036, -0.1235548 ],
       ...,
       [-0.15935828,  0.01698434, -0.01251688, ...,  0.124622  ,
        -0.11705713,  0.06674705],
       [-0.00379965, -0.01679246,  0.03485752, ...,  0.00597049,
        -0.04584568,  0.02619055],
       [ 0.01491513,  0.02812401, -0.01682792, ...,  0.15215527,
        -0.02477149,  0.01833932]], dtype=float32)

In [43]:
embeddings.shape

(20252, 768)

In [44]:
import faiss
vector_dimensions = embeddings.shape[1]

index = faiss.IndexFlatL2(vector_dimensions)
faiss.normalize_L2(embeddings)
index.add(embeddings)

In [45]:
import numpy as np

search_text = '≈y'
search_vector = encoder.encode(search_text)
new_vector = np.array([search_vector])
faiss.normalize_L2(new_vector)

In [47]:
distances,ann = index.search(new_vector,k=4)
results = pd.DataFrame({'distances': distances[0], 'ann': ann[0]})
df_merged = pd.merge(results,data,left_on='ann',right_index=True)

In [48]:
df_merged.head()

,distances,ann,title,title_sentiment
0,1.249892,10829,hit new resonac shwdy high 17.55 1 year otcmkts,2
1,1.282128,19124,energy 0.5 trade kos lon kosmos,0
2,1.300750,16169,share 1.7 etr brenntag bnr,0
3,1.318146,8044,pass two lon resource providence day move aver...,2


# Text Pre-processing
Tokenizing.

Lemmatizing.

Removal of stop words.

Removal of punctuations.

In [22]:
def preparation_text_data(data):

    # initialize spacy object
    nlp = spacy.load('en_core_web_sm')
    # select raw text
    raw_text = data.title.values.tolist()
    # tokenize
    tokenized_text = [[nlp(i.lower().strip())] for i in tqdm(raw_text)]
    #define the punctuations and stop words
    punc = string.punctuation
    stop_words = set(stopwords.words('english'))
    #lemmatize, remove stopwords and punctuationd
    corpus = []
    for doc in tqdm(tokenized_text):
        corpus.append([word.lemma_ for word in doc[0] if (word.lemma_ not in stop_words and word.lemma_ not in punc)])

    data["title"] = corpus
    return data

data =  preparation_text_data(data)

<ipython-input-22-9bad527feb5a>:8: TqdmDeprecationWarning:

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



  0%|          | 0/20252 [00:00<?, ?it/s]

<ipython-input-22-9bad527feb5a>:14: TqdmDeprecationWarning:

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



  0%|          | 0/20252 [00:00<?, ?it/s]

Sentiment Label Mapping

In [23]:
sentiment_mapping = {"Negative": 0, "Neutral": 1, "Positive": 2}

mask = data['title_sentiment'].isin(["Negative", "Neutral", "Positive"])

# Mapping of numerical predictions
data.loc[mask, 'title_sentiment'] = data.loc[mask, 'title_sentiment'].map(sentiment_mapping)

<ipython-input-23-d3294dee6851>:6: DeprecationWarning:

In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`



Text Representation

In [24]:
tfidf_vect = TfidfVectorizer()

def text_representation(data):
  data['title'] = data['title'].apply(lambda text: " ".join(set(text)))
  tfidf = tfidf_vect.fit_transform(data['title'])
  print(tfidf.shape)

  tfidf = pd.DataFrame(tfidf.toarray())
  return tfidf


tfidf = text_representation(data)

(20252, 21563)


In [25]:
tfidf

,0,1,2,3,4,5,6,7,8,9,...,21553,21554,21555,21556,21557,21558,21559,21560,21561,21562
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20247,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20248,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20249,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
# Feature selection
x= tfidf
y = data['title_sentiment'].astype('int')

In [30]:
# train test split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

# Model Training & Testing

In [31]:
clf= LogisticRegression()
clf.fit(x_train,y_train)
clf.score(x_test,y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



0.7621790651744569

In [32]:
# Prediction
y_pred = clf.predict(x_test)

In [33]:
# Classification Report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.78      0.87      0.83      2557
           1       0.68      0.61      0.65      1770
           2       0.80      0.76      0.78      1749

    accuracy                           0.76      6076
   macro avg       0.76      0.75      0.75      6076
weighted avg       0.76      0.76      0.76      6076



In [34]:
new_text = "texas instruments breaks ground on new 300-mm semiconductor wafer fabrication plant in utah"

# Vectorize the new text using the same TF-IDF vectorizer used for training
new_text_tfidf = tfidf_vect.transform([new_text])

# Make predictions using the trained logistic regression model
prediction = clf.predict(new_text_tfidf)

# Print the prediction
print("Predicted Sentiment:", prediction[0])

Predicted Sentiment: 1
